In [61]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, accuracy_score
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import pickle


# Constants
EEG_DATA_DIR = r'C:\Users\User\Documents\Lie detect data\56M_Test_DWTEEGData'
max_length = 1400  # Define maximum length for padding

# Define EEGNet model
class EEGNet(nn.Module):
    def __init__(self, num_classes=2):
        super(EEGNet, self).__init__()
        self.conv1 = nn.Conv1d(65, 32, kernel_size=63, padding=31)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.depthwiseConv1d = nn.Conv1d(32, 64, kernel_size=65, groups=32, padding=32)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.activation = nn.ReLU()
        
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)  # Additional convolutional layer
        self.batchnorm3 = nn.BatchNorm1d(128)
        
        self.pooling = nn.AvgPool1d(kernel_size=4)
        self.dropout = nn.Dropout(0.5)
        
        self.global_pool = nn.AdaptiveAvgPool1d(1)  # Global average pooling
        
        self._calculate_num_features()
        self.fc = nn.Linear(self.num_features, num_classes)

    def _calculate_num_features(self):
        with torch.no_grad():
            sample_input = torch.zeros(1, 65, 1400)
            sample_output = self._forward_features(sample_input)
            self.num_features = sample_output.shape[1]

    def _forward_features(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.activation(x)
        
        x = self.depthwiseConv1d(x)
        x = self.batchnorm2(x)
        x = self.activation(x)
        
        x = self.conv2(x)  # Additional convolutional layer
        x = self.batchnorm3(x)
        x = self.activation(x)
        
        x = self.pooling(x)
        x = self.dropout(x)
        x = self.global_pool(x)  # Global average pooling layer
        x = torch.flatten(x, 1)  # Flatten the output for the fully connected layer
        return x

    def forward(self, x):
        x = self._forward_features(x)
        x = self.fc(x)
        return x
        
# Function to load and label data (same as in your training script)
def load_data(data_dir, max_length):
    X = []
    y = []
    file_list = os.listdir(data_dir)
    for file in file_list:
        with open(os.path.join(data_dir, file), 'rb') as f:
            data = pickle.load(f)
        label = 1 if 'truth' in file else 0
        padded_data = np.zeros((65, max_length))
        length = min(data.shape[1], max_length)
        padded_data[:, :length] = data[:, :length]
        X.append(padded_data)
        y.append(label)
    return np.array(X), np.array(y)

# Define dataset class
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)


# Main script
if __name__ == "__main__":
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the saved model
    model_path = r'C:\Users\User\Documents\Lie detect data\Model\fold3_model_fold_4.pth'
    model = EEGNet(num_classes=2).to(device)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval

    # Move the model to the appropriate device
    model.to(device)

    # Load and preprocess the training data to create the scaler
    X, y = load_data(EEG_DATA_DIR, max_length)

    # Load the scaler
    with open(r'C:\Users\User\Documents\Lie detect data\Model\simpleEEGNet_scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    X = scaler.transform(X.reshape(X.shape[0], -1))
    X = X.reshape(-1, 65, max_length)
    

    test_dataset = EEGDataset(X, y)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    all_labels = []
    all_predictions = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        print(f"batch shape: {X_batch.shape}")
        
        X_batch = X_batch.to(device)
        labels = y_batch.to(device)
        
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Convert lists to numpy arrays for metric calculations
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_predictions)
conf_matrix = confusion_matrix(all_labels, all_predictions)


# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f'Precision: {precision}, Recall: {recall}, F1-score: {f1}, AUC: {auc}')
print('Confusion Matrix:')
print(conf_matrix)

    

batch shape: torch.Size([32, 65, 1400])
batch shape: torch.Size([32, 65, 1400])
batch shape: torch.Size([32, 65, 1400])
batch shape: torch.Size([24, 65, 1400])
Accuracy: 0.5917
Precision: 1.0, Recall: 0.4895833333333333, F1-score: 0.6573426573426573, AUC: 0.7447916666666666
Confusion Matrix:
[[24  0]
 [49 47]]
